In [16]:
import pandas as pd

In [ ]:
import os
import pandas as pd

# folder = "/Users/luisfaria/Desktop/sEngineer/masters_SWEAI/2025-T2/T2-REM/assignments/Assessment2/sales"

# If in codespaces
folder = "/workspaces/masters-swe-ai/2025-T2/T2-REM/assignments/Assessment2/sales"

# List to hold dataframes
dataframes = []

# Read all Excel files and append to list
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        filepath = os.path.join(folder, file)
        df = pd.read_excel(filepath)
        dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Data combined successfully.")
else:
    print("No Excel files found.")

# Optional: print the first few rows
combined_df.sample(5)

Data combined successfully.


,ID orçamento,Data orçamento,Status,Revenda,Data venda,Mês venda,Hora venda,Unidade,Consultor,Avaliador,...,Valor liquido item,Fonte do cadastro do cliente,Nome cliente,CPF cliente,ID cliente,Email do cliente,Telefone(s) do cliente,Data nascimento cliente,Profissão cliente,Cortesia?
170,423919,2025-01-03 10:16:06,Finalizado,SIM,2025-01-03 10:16:52,Janeiro,10:16:52,IPIRANGA,Beatriz Abrantes Duarte,Raphaela Serra Azevedo,...,502.0,Instagram,Denise Aparecida Meneguello,065.007.318-59,659120.0,denisemeneguello@gmail.com,11998214281,1964-12-07 00:00:00,Secretária,Não
3031,426983,2025-01-21 12:58:16,Finalizado,NÃO,2025-01-21 13:00:07,Janeiro,13:00:07,MOOCA,Vitoria Lins Ferreira,Amanda Santana dos Santos,...,150.0,Facebook Leads,Fernanda Miranda,263.269.488-43,1014534.0,fernanda@dtadvogados.com.br,11997049975,1978-10-18 00:00:00,Administrador (a),Não
15132,439402,2025-03-26 17:57:00,Finalizado,SIM,2025-03-26 18:14:03,Março,18:14:03,GUARULHOS,Thais Vieira Matias,AMANDA SOUZA FAIOLI GRU,...,890.0,SEMANA SEM RUGAS TATUAPÉ FACEBOOK,Marcia Cristina Rangel Colbo,086.916.118-04,426260.0,marcia.colbo@hotmail.com,11998225033,1954-06-30 00:00:00,Aposentado(a),Não
16890,441057,2025-03-31 20:26:18,Finalizado,NÃO,2025-03-31 20:30:41,Março,20:30:41,PRAIA GRANDE,Julia Sousa,NaN,...,990.0,Siluets Santos,Maristela Marques,271.052.611-53,602640.0,maristelamarquesrp@gmail.com,4885009244,1963-08-02 00:00:00,Outros,Não
16912,441072,2025-03-31 21:20:32,Finalizado,NÃO,2025-03-31 21:21:20,Março,21:21:20,ITAIM,Thais Silva Lima,NaN,...,397.0,Mídia Whatsapp,Diovanna Santana Caldeira Lima,457.394.678-02,996875.0,diovanna_lima97@hotmail.com,11982523978,1997-01-30 00:00:00,Analista,Não


In [18]:
df.columns

Index(['ID orçamento', 'Data orçamento', 'Status', 'Revenda', 'Data venda',
       'Mês venda', 'Hora venda', 'Unidade', 'Consultor', 'Avaliador',
       'Valor tabela', 'Valor desconto', 'Valor líquido', 'Grupo procedimento',
       'Procedimento', 'Quantidade', 'Valor tabela item',
       'Valor % desconto item', 'valor desconto item', 'Valor liquido item',
       'Fonte do cadastro do cliente', 'Nome cliente', 'CPF cliente',
       'ID cliente', 'Email do cliente', 'Telefone(s) do cliente',
       'Data nascimento cliente', 'Profissão cliente', 'Cortesia?'],
      dtype='object')

In [19]:
desired_columns = [
        'ID orçamento', 'Status','Data venda', 'Mês venda',
        'Unidade', 'Valor líquido', 'ID cliente']

df_filtered = combined_df[desired_columns]

# treating
df_filtered = df_filtered.loc[df_filtered['Status'] == 'Finalizado']
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].fillna(0)
df_filtered.sort_values(by='Valor líquido', ascending=False)

# total sum of 'Valor líquido'
df_filtered['Valor líquido'].sum()



np.float64(11038783.780000001)

In [21]:
# convert 'Valor líquido' to float
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].astype(float)

df_filtered.groupby(['Unidade', 'Mês venda'])['Valor líquido'].sum().unstack()

Mês venda,Fevereiro,Janeiro,Março
Unidade,,,
ALPHAVILLE,82787.04,76386.00,60769.91
CAMPINAS,81204.76,104370.99,185642.03
COPACABANA,264137.09,266988.73,285692.08
GUARULHOS,64337.99,61824.02,71547.97
IPIRANGA,177788.51,230115.80,239213.04
ITAIM,74078.00,129310.04,149389.01
JARDINS,437685.85,442856.93,601662.02
LAPA,250000.08,197947.89,186461.98
LONDRINA,141187.94,190215.03,200082.01
